In [ ]:
import tkinter as tk
import tkinter.filedialog as fd
from selenium.webdriver.common.by import By
# root = tk.Tk()
# file = fd.askopenfilename(parent=root, title='Choose a record file')
# file2 = fd.askopenfilenames(parent=root, title='Choose a pdf files')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output
# root.destroy()
from time import sleep
import easygui
import time
import re
from datetime import date
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
from selenium import webdriver
#set chromedriver.exe path
driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
driver.implicitly_wait(0)
#maximize browser
driver.maximize_window()
#launch URL
driver.get("https://www.amazon.in/Carrier-Flexicool-Inverter-Convertible-Filtration/dp/B0BR58HRD2/ref=sr_1_1_sspa?keywords=air%2Bconditioner&qid=1684858517&sprefix=air%2Bcond%2Caps%2C240&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1")


In [ ]:
cc=['LED Lamps','RAC (Fixed speed)','Direct cool refrigerator','Ceiling Fans']

list=['https://www.amazon.in/s?k=LED+Lamps&crid=I2MRZXKVJJQN&sprefix=led+lamps%2Caps%2C217&ref=nb_sb_noss_1',
'https://www.amazon.in/s?k=Air+Conditioners+Non+Inverter&crid=28QDARQC9SE0Q&sprefix=air+conditioners+non+inverter%2Caps%2C226&ref=nb_sb_noss_1',
'https://www.amazon.in/s?k=Single+Door+Refrigerators&crid=YDYG5X4C8OWU&sprefix=single+door+refrigerators%2Caps%2C206&ref=nb_sb_noss_1',
'https://www.amazon.in/s?k=Ceiling+Fans&crid=22QIH176VKOVS&sprefix=ceiling+fans%2Caps%2C224&ref=nb_sb_noss_1']

for w in range(len(list)):
    category=str(cc[w])


    #Get product link
    driver.get(str(list[w]))
    title=[]
    link=[]
    while True:
        try:
            time.sleep(1)
            job=driver.find_element(by=By.CLASS_NAME, value='sg-col-20-of-24')
            soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
            a=soup.find_all("a",class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
            for job_elem in a:
                link.append('https://www.amazon.in/'+job_elem['href'])
            c=soup.find_all("a",class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
            for job_elem in c:
                title.append(job_elem.text)
            b=soup.find("a",class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")['href']
            driver.get('https://www.amazon.in/'+b)
    #         print(len(link))
    #         print(len(title))
        except:
            break

    df = pd.DataFrame({
    'Title': title,
    'Link': link})
    df
    df.to_excel(str(category)+'.xlsx')

    df=pd.read_excel(str(category)+".xlsx")
#     len(df)

    #Get images link
    driver.implicitly_wait(0)
    About=[]
    prodDetails=[]
    mainimg=[]
    for j in range(0,len(df)):
        print(j)
        driver.get(df['Link'][j])
        try:
            a=driver.find_element(by=By.CLASS_NAME, value='centerColAlign ').text
            About.append(a)
        except:
            About.append('none')
        try:
            b=driver.find_element(by=By.ID, value='prodDetails').text
            prodDetails.append(b)
        except:
            prodDetails.append('none')
        try:
            try:
                cl=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div[5]/div[3]/div[3]/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/ul/li[1]/span/span/div')))
            except:
                cl=WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[2]/div[5]/div[3]/div[3]/div[1]/div[1]/div/div/div/div[1]/div[1]/ul/li[1]/span/span/div')))
            cl.click()                                                               
            time.sleep(1)
            img=[]
            e=driver.find_elements(by=By.CLASS_NAME, value='ivThumb')
            for x in range(2,len(e)-1):
                try:
                    e[x].click()
                    time.sleep(0.4)
                    job=driver.find_element(by=By.ID, value='ivLargeImage')
                    soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
                    a=soup.find_all("img",class_="fullscreen")
                    for item in a:
                        img.append(item['src'])
                        print(item['src'])
                except:
                    pass
        except:
            img.append('none')
        mainimg.append(img)

    df['About']=0
    df['image ink']=0
    df['prodDetails']=0
    df['About']=About
    df['prodDetails']=prodDetails
    df['image ink']=mainimg
    df.to_csv('final'+str(category)+'.csv')